In [1]:
import requests
import re
import json

house_file = '/Users/oliver/Documents/wikipedia_house.html'
senate_file = '/Users/oliver/Documents/wikipedia_senate.html'

In [2]:
# parse html to build congress member dictionaries

house = ''
senate = ''

with open(house_file) as d:
    house = ''.join(d.readlines())
    
with open(senate_file) as d:
    senate = ''.join(d.readlines())
    
house = re.findall(r'<span class="fn">(.*?)</a></span>', house, re.DOTALL)
senate = re.findall(r'<span class="fn">(.*?)</a></span>', senate, re.DOTALL)

house_info = []
senate_info = []

for x in house:
    wiki = x.split('<a href="')[1].split('" title')[0]
    name = x.split('">')[1]
    house_info.append({ 'url': wiki, 'name': name})
    
for x in senate:
    wiki = x.split('<a href="')[1].split('" title')[0]
    name = x.split('">')[1]
    senate_info.append({ 'url': wiki, 'name': name})

In [3]:
def crawl(url):
    full_url = 'http://en.wikipedia.org' + url
    text = requests.get(full_url).text
    text = text.encode('utf-8')
    text = re.sub("\xe2\x80\x93", "-", text)
    text = re.sub('\[[0-9]\]', '', text)
    try:
        intro = text.split('infobox vcard')[1].split('</table>')[1].split('<div id="toc"')[0]
    except Exception:
        return ''
    notag = re.sub("<.*?>", " ", intro).strip()
    notag = notag.replace('\n', '')
    notag = notag.replace(" \'s", "'s")
    notag = notag.replace('   ', " ")
    notag = notag.replace('  ', " ")
    notag = notag.replace('  ', " ")
    notag = notag.replace(" .", ".")
    notag = notag.strip()
    return notag

In [4]:
# crawl congress member wikipedia pages

for i in range(0, len(senate_info) - 1):
    url = senate_info[i]['url']
    text = crawl(url)
    row = { 'name': senate_info[i]['name'], 'url': senate_info[i]['url'], 'biography': text}
    senate_info[i] = row
    
for i in range(0, len(house_info) - 1):
    url = house_info[i]['url']
    text = crawl(url)
    row = { 'name': house_info[i]['name'], 'url': house_info[i]['url'], 'biography': text}
    house_info[i] = row

In [5]:
# write to text file

json.dump(senate_info, open("senate_biographies.txt",'w'))
json.dump(house_info, open("house_biographies.txt",'w'))